In [ ]:
!pip install -U spacy
!pip install -U gensim

     |████████████████████████████████| 6.0 MB 8.2 MB/s 
     |████████████████████████████████| 10.1 MB 50.3 MB/s 
     |████████████████████████████████| 628 kB 56.3 MB/s 
     |████████████████████████████████| 42 kB 959 kB/s 
     |████████████████████████████████| 181 kB 65.6 MB/s 
     |████████████████████████████████| 451 kB 49.2 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 24.1 M

# **Import libraries**

In [ ]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR, StepLR

from torchtext.vocab import Vocab, vocab
from torchtext.vocab import build_vocab_from_iterator
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
pio.renderers.default = 'colab'

import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
import random
from datetime import datetime
from pathlib import Path
from scipy.sparse import hstack
from collections import Counter, OrderedDict

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Token

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim

In [ ]:
# Install wandb and update it to the latest version
%%capture
!pip install wandb --upgrade

In [ ]:
# Import wandb
import wandb

# Login to W&B
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## **Import Dataset csv file**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# specify the folder in googe drive where we will save dataset
basepath = '/content/drive/MyDrive/Data'

In [ ]:
data_folder = Path('/content/drive/MyDrive/NLP/Homework21')

In [ ]:
spacy_folder = Path('/content/drive/MyDrive/Data/spacy')

In [ ]:
lecture_folder = Path('/content/drive/MyDrive/NLP')

In [ ]:
save_model_folder = lecture_folder /'Homework21'

In [ ]:
#url = 'https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.2.0/en_core_web_lg-3.2.0.tar.gz'
#!wget {url} -P {spacy_folder} 

In [ ]:
#file = spacy_folder / 'en_core_web_lg-3.2.0.tar.gz'
#with  tarfile.open(file, 'r') as tar:
#  tar.extractall(path = spacy_folder)

In [ ]:
model = spacy_folder /'en_core_web_lg-3.2.0'/'en_core_web_lg'/'en_core_web_lg-3.2.0'
nlp = spacy.load(model, disable=['ner, parser'])

In [ ]:
folder = Path(basepath)
folder

PosixPath('/content/drive/MyDrive/Data')

In [ ]:
df = pd.read_csv(folder / 'multilabel_hw.csv')
df.drop(columns= ['Unnamed: 0','Unnamed: 0.1','Id'], axis=1, inplace=True)
df

,Title,Body,Tags,Tag_Number
0,ASP Query String From DropDown,<p>I have a webpage: <strong>Menu.aspx</strong...,c# asp.net,"[0, 9]"
1,How can I run JavaScript code at server side J...,<p>I want to run JavaScript code at the server...,java javascript,"[1, 3]"
2,linq to sql throwing an exception row not foun...,<p>Hi I am linq to sql and i am getting the er...,c# asp.net,"[0, 9]"
3,Running a Python script on a PHP server,"<p>I am running a nginx web server, along with...",php python,"[2, 7]"
4,some advice on how to write a window.resize fu...,<p>Im trying to write a function that resizes ...,javascript jquery,"[3, 5]"
...,...,...,...,...
47422,Taking value of edittext and putting a decimal...,<p>All-\nI am working on an app in which the u...,java android,"[1, 4]"
47423,Listen to phone state throughout the application?,<p>I nee to liste to phone state using phone s...,java android,"[1, 4]"
47424,Android UI Thread,<p>i am using threads to do few tasks. and aft...,java android,"[1, 4]"
47425,dynamic table row creation in html/Javascript,<p>I have html table with 1 row to fill in job...,asp.net javascript,"[9, 3]"


In [ ]:
mlb = MultiLabelBinarizer()
df1 = pd.DataFrame(mlb.fit_transform(df['Tag_Number']),columns=mlb.classes_)
cols = [0,1,12,13]
df1.drop(df1.columns[cols],axis=1,inplace=True)
print (df1)

       0  1  2  3  4  5  6  7  8  9
0      1  0  0  0  0  0  0  0  0  1
1      0  1  0  1  0  0  0  0  0  0
2      1  0  0  0  0  0  0  0  0  1
3      0  0  1  0  0  0  0  1  0  0
4      0  0  0  1  0  1  0  0  0  0
...   .. .. .. .. .. .. .. .. .. ..
47422  0  1  0  0  1  0  0  0  0  0
47423  0  1  0  0  1  0  0  0  0  0
47424  0  1  0  0  1  0  0  0  0  0
47425  0  0  0  1  0  0  0  0  0  1
47426  0  0  0  1  0  1  0  0  0  0

[47427 rows x 10 columns]


In [ ]:
df = pd.concat([df,df1],axis=1)

In [ ]:
X = df['Body'][10000:20000].values #drop(columns = ['Body','Tag','Tag_Number'])

y = df[['0','1','2','3','4','5','6','7','8','9']][10000:20000].values.astype(float)  #'0','1','2','3','4','5','6','7','8','9'

# Create train/test/valid
splits from dataset. 20000 for train, 20000 for test, remaining for validation dataset

In [ ]:
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.6,random_state=42)

In [ ]:
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.2,random_state=42)

In [ ]:
print(X_train.shape), print(y_train.shape)
print(X_test.shape), print(y_test.shape)
print(X_valid.shape), print(y_valid.shape)


(6000,)
(6000, 10)
(800,)
(800, 10)
(3200,)
(3200, 10)


(None, None)

## PreProcess Data

In [ ]:
class SpacyPreprocessor(BaseEstimator, TransformerMixin):
    np.random.seed(0)
    def __init__(self, lammetize=True, lower=True, remove_stop=True, remove_punct=True, remove_num=False):
        self.remove_stop = remove_stop
        self.remove_punct = remove_punct
        self.remove_num = remove_num
        self. lammetize = lammetize
        self.lower = lower

    # helpfer functions for basic cleaning 
    def basic_clean(self,text):
        return [re.sub(r'[\n\r]',' ',sentence) for sentence in text]

    # helper function for pre-processing with spacy
    def spacy_preprocessor(self,texts): 
        nlp=spacy.load(model, disable=['parser','ner'])
        ## Add @ as a prefix so that we can separate the word from its token
        ## Since we are using pretrained vectors - @ mentions will be different in the pre-trained vocab
        
        prefixes = list(nlp.Defaults.prefixes)
        prefixes += ['@']
        prefix_regex = spacy.util.compile_prefix_regex(prefixes)
        nlp.tokenizer.prefix_search = prefix_regex.search
     
        matcher = Matcher(nlp.vocab)
        if self.remove_stop:
            matcher.add("stop_words", [[{"is_stop" : True}]])
        if self.remove_punct:
            matcher.add("punctuation",[ [{"is_punct": True}]])
        if self.remove_num:
            matcher.add("numbers", [[{"like_num": True}]])
        Token.set_extension('is_remove', default=False,force=True)
        cleaned_text=[]

        for doc in nlp.pipe(texts,batch_size=64,disable=['parser','ner']):
            matches = matcher(doc)
            for _, start, end in matches:
                for token in doc[start:end]:
                    token._.is_remove =True
                    
            if self.lammetize:
                text = ' '.join(token.lemma_ for token in doc if (token._.is_remove==False))
            else:
                text = ' '.join(token.text for token in doc if (token._.is_remove==False))
            if self.lower:
                text=text.lower()
            cleaned_text.append(text)
        return cleaned_text

    def fit(self, X,y=None):
        return self

    def transform(self, X,y=None):
        x_clean = self.basic_clean(X)
        x_clean_final = self.spacy_preprocessor(x_clean)
        return x_clean_final

In [ ]:
#preprocessor = SpacyPreprocessor()

In [ ]:
#X_train_cleaned = preprocessor.fit_transform(X_train)

In [ ]:
#X_valid_cleaned = preprocessor.transform(X_valid)

In [ ]:
#X_test_cleaned = preprocessor.transform(X_test)

In [ ]:
#X_train_cleaned = np.array(X_train_cleaned)
#X_valid_cleaned = np.array(X_valid_cleaned)
#X_test_cleaned = np.array(X_test_cleaned)

In [ ]:
file_X_train_cleaned_data = data_folder/ 'X_train_multiclass_clean_task5.pkl'

In [ ]:
file_X_valid_cleaned_data = data_folder/ 'X_valid_multiclass_clean_task5.pkl'

In [ ]:
file_X_test_cleaned_data = data_folder/'X_test_multiclass_clean_task.pkl'

In [ ]:
#joblib.dump(X_train_cleaned, file_X_train_cleaned_data) 
#joblib.dump(X_valid_cleaned, file_X_valid_cleaned_data) 
#joblib.dump(X_test_cleaned, file_X_test_cleaned_data) 

In [ ]:
X_train_cleaned = joblib.load(file_X_train_cleaned_data)
X_valid_cleaned = joblib.load(file_X_valid_cleaned_data)
X_test_cleaned = joblib.load(file_X_test_cleaned_data)

In [ ]:
print(X_train_cleaned.shape)
print(X_test_cleaned.shape)
print(X_valid_cleaned.shape)

(6000,)
(800,)
(3200,)


## Custom Dataset Class

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """IMDB dataset."""

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        text = self.X[idx]
        text = np.array(text)
        labels = self.y[idx]
        labels = np.array(labels)
        sample = (labels,text)
        
        return sample

In [ ]:
trainset = CustomDataset(X_train_cleaned,y_train)
validset = CustomDataset(X_valid_cleaned,y_valid)
testset = CustomDataset(X_test_cleaned,y_test)

In [ ]:
trainset

In [ ]:
trainset.__getitem__([0,2])

(array([[1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 array(['< p > way embed user control windows explorer tell resource < /p >   < p > thanks weipeng</p >',
        '< p > need help copde datum table manually create row column dt populate lastly bind gridview.</p >   < p > want add cell datarow datum table cell turn hold html control html anchor tag < /p >   < p > current dt 2rows 3 col below</p >   < pre><code > server   blah   blah abc      xyz    123 def      vbh    345 < /code></pre >   < p > want processing servername col1 add extra col dt hold html anchor tag detail see click html anchor tag ultimately dt look like below:</p >   < pre><code > server   blah   blah   abc      xyz    123    html link def      vbh    345    html link < /code></pre >   < p > tell i.e add separate cell dt add html control bewly add cell.</p >   < p > thanks rahul</p >'],
       dtype='<U32849'))

## Small Sample

In [ ]:
# We will be using only 100 images for nboth train and validation datasets
train_sample_size = int(len(trainset)/10)
valid_sample_size = int(len(validset)/7)

# Getting n random indices
train_subset_indices = random.sample(range(0, len(trainset)), train_sample_size)
valid_subset_indices = random.sample(range(0, len(validset)), valid_sample_size)

# Getting subset of dataset
train_subset = torch.utils.data.Subset(trainset, train_subset_indices)
valid_subset = torch.utils.data.Subset(validset, valid_subset_indices)

In [ ]:
# craete a train and test dataloader
#train_subset = CustomDataset(X_train_cleaned[train_subset_indices],y_train[train_subset_indices])
                          
#valid_subset = CustomDataset(X_valid_cleaned[valid_subset_indices],y_valid[valid_subset_indices])

## Create Vocab

In [ ]:
counter = Counter()
for (label, line) in trainset:
   counter.update(str(line).split())

In [ ]:
#Sorting the words based on their frequency and creating OrderedDict from it in descending order
sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab_dict = vocab(counter, min_freq=2)

In [ ]:
#vocab_dict.get_stoi()

In [ ]:
len(vocab_dict)

25039

In [ ]:
vocab_dict.insert_token('<unk>', 0)

In [ ]:
vocab_dict.set_default_index(0)

## Collate_fn for Data Loaders

In [ ]:
# Creating a lambda function objects that will be used to get the indices of words from vocab
text_pipeline = lambda x: [vocab_dict[token] for token in str(x).split()]
label_pipeline = lambda x: [int(i) for i in x]


In [ ]:
'''
We know that input to the embedding layers are indices of words from the vocab.
The collate_batch() accepts batch of data and gets the indices of text from vocab and returns the same
We will include this collate_batch() in collat_fn attribute of DataLoader.
So it will create a batch of data containing indices of words and corresponding labels.
But for EmbeddingBag we need one more extra parameter, that is offset.
offsets determines the starting index position of each bag (sequence) in input.
'''
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list, label_list,  offsets

## **Check Data Loader**



In [ ]:
batch_size=2
check_loader= torch.utils.data.DataLoader(dataset=trainset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        collate_fn=collate_batch,
                                        num_workers=2)

In [ ]:
for label, text, offsets in check_loader:
  print(label, text, offsets)
  break

tensor([    1,   259,     3,     1,   260,     3,   261,   262,     3,     1,
          263,     0,   409,   140,   678,   745,   756,   840,     0,   409,
          140,   678,   745,     0,   840,     0,     3,     1,    12,     3,
            1,   268,     3,     1,     2,     3,   269,   646,   140,    39,
         2760,  2161,   255,   399,   109,    42,  4471,   289,   140,   678,
         3261,   251,   678,  2092,   745,   544,  1105,    39,  1096,   616,
          824,   140,  6034,   680,   164,  1041,   589,   140,  1063,   329,
          473,   678,   745,   255,   586,   140,    34,    35,   745,    28,
          778,   678,  1956,   131,   327,  2795,  1508, 14554,     3,     1,
          578,   263,     0,   255,  4918,  2092,   136,     0,   255,  4918,
         2092,   136,     0,     3,     1,     2,     3,   329,  2510,     3,
            1,    68,     3,     0,     1,    40,     3,     1,     2,     3,
          109,  1950,   251, 13653,     3,     1,     2,     3, 

# Create Weight Matrix of Pretrained Weights

In [ ]:
pretrained_vectors = KeyedVectors.load('/content/drive/MyDrive/NLP/Homework21/model_df.bin')

In [ ]:
vector = pretrained_vectors.get_vector('javascript', norm=True)
vector.shape

(150,)

In [ ]:
embedding_dim =150
test_weights = np.zeros((2, embedding_dim))

In [ ]:
test_weights[0] = pretrained_vectors.get_vector('javascript', norm=True)

In [ ]:
test_weights[1] =  np.random.normal(size=(embedding_dim, ))

In [ ]:
test_weights

array([[-8.49839970e-02, -2.17764992e-02, -6.82247356e-02,
         5.70525881e-03,  2.65885089e-02,  1.32282272e-01,
        -4.59169894e-02, -2.73072417e-03,  5.12668304e-02,
        -9.32804793e-02,  2.14768946e-02, -5.41905826e-03,
        -6.06425386e-03,  1.36130795e-01,  1.90235917e-02,
         1.46427318e-01,  5.81797771e-02, -6.00542547e-03,
         7.31237000e-03, -6.35026023e-02,  5.12750894e-02,
        -7.50809442e-04,  1.86569560e-02, -7.20354319e-02,
        -7.38556683e-02,  1.22542307e-01,  3.26105095e-02,
         3.38960290e-02,  1.43136814e-01,  1.45177590e-02,
         9.68288779e-02,  5.26264310e-02, -5.18140336e-03,
        -2.11011129e-03, -1.56064387e-02, -7.48816431e-02,
        -1.10546961e-01,  9.78591442e-02, -5.69242612e-02,
         1.93735644e-01,  2.65050828e-02, -9.60124657e-03,
         7.94634297e-02,  3.92516293e-02, -2.60088556e-02,
         1.00974530e-01, -8.80348682e-02,  1.30057856e-01,
        -4.09416333e-02, -1.32559299e-01,  4.95028086e-0

In [ ]:
len(vocab_dict)

25040

In [ ]:
embedding_dim = 150
pretrained_weights = np.zeros((len(vocab_dict), embedding_dim))
words_found = 0
words_not_found = 0

for i, word in enumerate(vocab_dict.get_itos()):
    try: 
        pretrained_weights[i] = pretrained_vectors.get_vector(word, norm=True)
        words_found += 1
    except KeyError:
        words_not_found  += 1
        pretrained_weights[i] = np.random.normal(size=(embedding_dim, ))

In [ ]:
words_found

25040

In [ ]:
words_not_found

0

# Model 

In [ ]:
# Define custom model using nn.Module()
class MLPCustom2(nn.Module):
  def __init__(self, vocab_size, h_sizes_list, dprobs_list, batchnorm_binary, output_dim, non_linearity, pretrained_weights, task):
        
    super().__init__()

    self.h_sizes_list = h_sizes_list # h_sizes = [emb_dim, hidden_dim1,....hidden_dim2,....hidden_dimn] # n + 1 elements
    
    self.dprobs_list = dprobs_list # dpropb =[prob1, prob2....probn] # n elements
    self.batchnorm_binary = batchnorm_binary # [True, False....] # n elements

    
    self.non_linearity = non_linearity
    self.output_dim = output_dim
    self.vocab_size = vocab_size
    self.pretrained_weights = pretrained_weights
    self.task = task

    # Initialize hidden layers  

    self.hidden = nn.ModuleList()
    self.dropout = nn.ModuleList()
    self.batchnorm = nn.ModuleList()

    if self.task == 2:
       self.embedding = nn.EmbeddingBag(vocab_size, self.h_sizes_list[0])

    # Task 5
    if self.task == 5:
       self.embedding = nn.EmbeddingBag(vocab_size, self.h_sizes_list[0]).from_pretrained(pretrained_weights,
                                                                               freeze = True)
       
    # Task 6
    if self.task == 6:
       self.embedding = nn.EmbeddingBag(vocab_size, self.h_sizes_list[0]).from_pretrained(pretrained_weights,
                                                                               freeze = False)

    for k in range(len(h_sizes_list)-1):
      self.hidden.append(nn.Linear(self.h_sizes_list[k], h_sizes_list[k+1]))
      self.dropout.append(nn.Dropout(p=dprobs_list[k]))

      if self.batchnorm_binary:
        self.batchnorm.append(nn.BatchNorm1d(self.h_sizes_list[k+1], momentum=0.9))
      
    
    self.output_layer = nn.Linear(self.h_sizes_list[-1], output_dim)
   
    
    ## it is better to use nn.functional.relu in the forward function
    # self.relu = nn.ReLU()

  def forward(self, input, offsets):
    x = self.embedding(input, offsets)
    for  k in range(len(self.h_sizes_list)-1):
      x =  self.non_linearity(self.hidden[k](x))
      if self.batchnorm_binary:
        x = self.batchnorm[k](x)
      x= self.dropout[k](x)

    x = self.output_layer(x)
    # we are not using softmax function in the forward passs
    # nn.crossentropy loss (which we will use to define our loss) combines  nn.LogSoftmax() and nn.NLLLoss() in one single class
    return x  

# Training Functions

## Training Epoch 

In [ ]:
def train(train_loader, model, optimizer, loss_function, log_batch, log_interval, grad_clipping, max_norm):

  """ 
  Function for training the model in each epoch
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate.
  Output: final weights, bias, train loss, train accuracy
  """
  # initilalize variables as global
  # these counts will be updated every epoch
  global example_ct_train
  global batch_ct_train

  # Training Loop loop
  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # put the model in training mode
  model.train()

  # Iterate on batches from the dataset using train_loader
  for input, targets, offsets in train_loader:
    
    # move inputs and outputs to GPUs
    input = input.to(device)
    targets = targets.to(device).float()
    offsets = offsets.to(device)

    # Forward pass
    output = model(input,offsets)
    loss = loss_function(output, targets)

    # Correct prediction
    # Correct prediction
    #Sigmoid = nn.Sigmoid()
    #output = Sigmoid(output) 
    #threshold = torch.tensor([0.5]).to(device)
    #print('output of sigmoitd',output)
    y_pred = (output>0).float()
    #print('predicted value',y_pred)
    #y_pred = torch.argmax(output, dim = 1)
    #correct = torch.sum(y_pred == targets)          #correct = sum_pred(y_pred,targets)                                      #
    #print('Correct value',correct)

    example_ct_train +=  len(targets)
    batch_ct_train += 1

    # set gradients to zero 
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Gradient Clipping
    if grad_clipping:
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm, norm_type=2)

    # Update parameters using their gradient
    optimizer.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    #running_train_correct += correct

    # log batch loss and accuracy
    if log_batch:
      if ((batch_ct_train + 1) % log_interval) == 0:
        wandb.log({f"Train Batch Loss  :": loss})
        #wandb.log({f"Train Batch Acc :": correct/(len(targets)*10)})

  
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)



  # Calculate accuracy for the whole dataset for a particular epoch
  #train_acc = running_train_correct/(len(train_loader.dataset)*10)

  return train_loss #train_acc

## Validation/Test Epoch

In [ ]:
def valid(loader, model, optimizer, loss_function, log_batch, log_interval):

  """ 
  Function for training the model and plotting the graph for train & valid loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and valid loss for each epoch.
  """

  # initilalize variables as global
  # these counts will be updated every epoch
  global example_ct_valid
  global batch_ct_valid

  # Validation loop
  # Initialize train_loss at the he strat of the epoch
  running_valid_loss = 0
  running_valid_correct = 0
  
  # put the model in evaluation mode
  model.eval()

  with torch.no_grad():
    for input,targets, offsets in loader:

      # move inputs and outputs to GPUs
      input = input.to(device)
      targets = targets.to(device).float()
      offsets = offsets.to(device)

      # Forward pass
      output = model(input,offsets)
      
      loss = loss_function(output,targets)

      # Correct Predictions
      # Correct prediction
      #Sigmoid = nn.Sigmoid()
      #output = Sigmoid(output) 
      #threshold = torch.tensor([0.5]).to(device)
      #print('output of sigmoitd',output)
      y_pred = (output>0).float()
      #print('predicted value',y_pred)
      #y_pred = torch.argmax(output, dim = 1)
      #correct = torch.sum(y_pred == targets)          #correct = sum_pred(y_pred,targets)                                      #
      #print('Correct value',correct)

      # count of images and batches
      example_ct_valid +=  len(targets)
      batch_ct_valid += 1

      # Add valid loss of a batch 
      running_valid_loss += loss.item()

      # Add correct count for each batch
      #running_valid_correct += correct

      # log batch loss and accuracy
      if log_batch:
        if ((batch_ct_valid + 1) % log_interval) == 0:
          wandb.log({f"Valid Batch Loss  :": loss})
          #wandb.log({f"Valid Batch Accuracy :": correct/(len(targets)*10)})


    # Calculate mean valid loss for the whole dataset for a particular epoch
    valid_loss = running_valid_loss/len(valid_loader)

    # scheduler step
    # scheduler.step(valid_loss)
    # scheduler.step()

    # Calculate accuracy for the whole dataset for a particular epoch
    #valid_acc = running_valid_correct/(len(valid_loader.dataset)*10)
    
  return valid_loss #valid_acc

##  Model Training Loop


In [ ]:
def train_loop(train_loader, valid_loader, model, loss_function, optimizer, epochs, device, patience, early_stopping,
               file_model):

  '''
  model: specify your model for training
  criterion: loss function 
  optimizer: optimizer like SGD , ADAM etc.
  train loader: function to carete batches for training data
  loader : function to create batches for valid data set
  file_model : specify file name for saving your model. This way we can upload the model weights from file. We will not to run model again.
  

  '''
  # Create lists to store train and valid loss at each epoch

  train_loss_history = []
  valid_loss_history = []
  #train_acc_history = []
  #valid_acc_history = []
  delta = 0
  best_score = None
  valid_loss_min = np.Inf
  counter_early_stop=0
  early_stop=False


  # Iterate for the given number of epochs
  for epoch in range(epochs):
    t0 = datetime.now()
    # Get train loss and accuracy for one epoch

    train_loss = train(train_loader, model, optimizer, loss_function, 
                                  wandb.config.log_batch, wandb.config.log_interval,
                                  wandb.config.grad_clipping, wandb.config.max_norm)
    valid_loss = valid(valid_loader, model, optimizer, loss_function,
                                    wandb.config.log_batch, wandb.config.log_interval)

    dt = datetime.now() - t0

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    #train_acc_history.append(train_acc)
    valid_loss_history.append(valid_loss)
    #valid_acc_history.append(valid_acc)

    if early_stopping:
      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        counter_early_stop += 1
        print(f'Early stoping counter: {counter_early_stop} out of {patience}')
        if counter_early_stop > patience:
          early_stop = True

      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        counter_early_stop=0
        valid_loss_min = valid_loss

      if early_stop:
        print('Early Stopping')
        break

    else:

      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        print(f'Validation loss has not decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Not Saving Model...')
      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss



    # Log the train and valid loss to W&B
    wandb.log({f"Train epoch Loss :": train_loss, f"Valid epoch Loss :": valid_loss })
    #wandb.log({f"Train epoch Acc :": train_acc, f"Valid epoch Acc :": valid_acc})



    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Time to complete {epoch+1} is {dt}')
    # print(f'Learning rate: {scheduler._last_lr[0]}')
    print(f'Train Loss: {train_loss : .4f} | Train Accuracy: ')
    print(f'Valid Loss: {valid_loss : .4f} | Valid Accuracy: ')
    print()
    torch.cuda.empty_cache()

  return train_loss_history, valid_loss_history #, valid_acc_history,train_acc_history,


#Model Training

In [ ]:
hyperparameters = dict(
    
    h_sizes_list = [150] + [200], # 300 = embed_dim
    dprobs_list = [0],
    batchnorm_binary = False,
    task = 6,
    vocab_size = len(vocab_dict),
    output_dim = 10,
    epochs = 20,
    batch_size = 128,
    learning_rate = 0.01,
    dataset="Multilabel",
    architecture="MLP",
    log_interval = 25,
    log_batch = True,
    file_model = save_model_folder/'regular_exp1.pt',
    grad_clipping = True,
    max_norm = 1,
    momentum = 0,
    patience = 10,
    early_stopping = True,
    scheduler_factor = 0.5,
    scheduler_patience = 0,
    weight_decay = 0.002
   )

# non_linearity = F.elu 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
non_linearity = F.selu
pretrained_weights_tensor = torch.tensor(pretrained_weights).float()

In [ ]:
pretrained_weights_tensor.shape

torch.Size([25040, 150])

In [ ]:
wandb.init(name = 'shishir_hw4_Q5_exp2', project = 'shishir_hw4_nlp', config = hyperparameters)

wandb: Currently logged in as: sxs200126 (use `wandb login --relogin` to force relogin)


In [ ]:
# wandb.config.non_linearity = non_linearity
wandb.config.device = device
print(wandb.config.device )
wandb.config.non_linearity = non_linearity
print(wandb.config.non_linearity)
# print(wandb.config.non_linearity)

cuda:0
torch.nn.functional.selu


## Specify Dataloader, Loss_function, Model, Optimizer, Weight Initialization

In [ ]:
# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Data Loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=wandb.config.batch_size, 
                                           shuffle = True, collate_fn=collate_batch, num_workers=2)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.batch_size, 
                                           shuffle = False, collate_fn=collate_batch, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.batch_size,   
                                         shuffle = False, collate_fn=collate_batch, num_workers=2)

# cross entropy loss function
loss_function = nn.BCEWithLogitsLoss()

# model 
model = MLPCustom2(wandb.config.vocab_size, wandb.config.h_sizes_list, wandb.config.dprobs_list, wandb.config.batchnorm_binary, 
           wandb.config.output_dim, non_linearity, pretrained_weights_tensor, wandb.config.task)

def init_weights(m):
  if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)

  if type(m) == nn.EmbeddingBag:
    torch.nn.init.normal_(m.weight, mean = 0, std = 1)

  

# apply init function recursibvely to all the modules
#model.apply(init_weights)

# put model to GPUs
model.to(device)

#Intialize stochiastic gradient descent optimizer
#optimizer = torch.optim.SGD(model.parameters(), lr = wandb.config.learning_rate, weight_decay=wandb.config.weight_decay, 
#                          momentum=wandb.config.momentum)

optimizer = torch.optim.Adam(model.parameters(), lr = wandb.config.learning_rate, weight_decay=wandb.config.weight_decay)

wandb.config.optimizer = optimizer



#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor= wandb.config.scheduler_factor, 
#                              patience=wandb.config.scheduler_patience, verbose=True)

# scheduler = StepLR(optimizer, gamma=0.4,step_size=1, verbose=True)

## Train Model and Save best model

In [ ]:
wandb.watch(model, log = 'all', log_freq=25, log_graph=True)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [ ]:
example_ct_train, batch_ct_train, example_ct_valid, batch_ct_valid = 0, 0, 0, 0
train_loss_history, valid_loss_history  = train_loop(train_loader, valid_loader, model, loss_function, optimizer, 
                                                                                          wandb.config.epochs, wandb.config.device,
                                                                                          wandb.config.patience, wandb.config.early_stopping,
                                                                                          wandb.config.file_model)

Validation loss has decreased (inf --> 0.382592). Saving Model...
Epoch : 1 / 20
Time to complete 1 is 0:00:02.573183
Train Loss:  0.4151 | Train Accuracy: 
Valid Loss:  0.3826 | Valid Accuracy: 

Validation loss has decreased (0.382592 --> 0.313292). Saving model...
Epoch : 2 / 20
Time to complete 2 is 0:00:02.490774
Train Loss:  0.3341 | Train Accuracy: 
Valid Loss:  0.3133 | Valid Accuracy: 

Validation loss has decreased (0.313292 --> 0.306464). Saving model...
Epoch : 3 / 20
Time to complete 3 is 0:00:02.436871
Train Loss:  0.3101 | Train Accuracy: 
Valid Loss:  0.3065 | Valid Accuracy: 

Early stoping counter: 1 out of 10
Epoch : 4 / 20
Time to complete 4 is 0:00:02.463132
Train Loss:  0.3091 | Train Accuracy: 
Valid Loss:  0.3098 | Valid Accuracy: 

Validation loss has decreased (0.306464 --> 0.302874). Saving model...
Epoch : 5 / 20
Time to complete 5 is 0:00:02.394326
Train Loss:  0.3094 | Train Accuracy: 
Valid Loss:  0.3029 | Valid Accuracy: 

Early stoping counter: 1 out of

# **Accuracy and Predictions**

Now we have final values for weights and bias after training the model. We will use these values to make predictions on the test dataset.

## Function to get predictions

In [ ]:
def get_acc_pred(data_loader, model):
  """ 
  Function to get predictions for a given test set and calculate accuracy.
  Input: Iterator to the test set.
  Output: Prections and Accuracy for test set.
  """
  with torch.no_grad():
    # Array to store predicted labels
    predictions = torch.Tensor()
    predictions = predictions.to(device)

    # Array to store actual labels
    y = torch.Tensor()
    y = y.to(device)
    # Iterate over batches from test set
    for input, targets in data_loader:
      
      # move inputs and outputs to GPUs
      input = input.to(device)
      targets = targets.to(device)

      # Calculated the predicted labels
      output = model(input)

      # Choose the label with maximum probability
      indices = torch.argmax(output, dim = -2)

      # Add the predicted labels to the array
      predictions = torch.cat((predictions, indices)) 

      # Add the actual labels to the array
      y = torch.cat((y, targets)) 

    # Check for complete dataset if actual and predicted labels are same or not
    # Calculate accuracy
    acc = (predictions == y).float().mean()

  # Return array containing predictions and accuracy
  return predictions, acc
  

## Load saved model from file 